<a href="https://colab.research.google.com/github/ssyeon2/kt-aivle/blob/main/DL(dropout%2C_BatchN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat

--2023-03-13 13:20:00--  http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat
Resolving yaroslavvb.com (yaroslavvb.com)... 129.121.4.193
Connecting to yaroslavvb.com (yaroslavvb.com)|129.121.4.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117586976 (112M)
Saving to: ‘notMNIST_small.mat.1’

notMNIST_small.mat. 100%[===================>] 112.14M  46.8MB/s    in 2.4s    

2023-03-13 13:20:02 (46.8 MB/s) - ‘notMNIST_small.mat.1’ saved [117586976/117586976]



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import io

In [ ]:
data = io.loadmat("notMNIST_small.mat")

In [ ]:
x = data['images']
y = data['labels']

- reshape

In [ ]:
x.shape, y.shape

((28, 28, 18724), (18724,))

In [ ]:
X = np.transpose(x, (2, 0, 1))
x = X.reshape(X.shape[0], 28, 28, 1)

In [ ]:
x.shape, y.shape

((18724, 28, 28, 1), (18724,))

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size= 0.2)

- scaling

In [ ]:
min_n, max_n = train_x.min(), train_x.max()
train_x = (train_x - min_n) /(max_n - min_n)
test_x = (test_x - min_n)/(max_n - min_n)

In [ ]:
train_x.min(), test_x.max()

(0.0, 1.0)

- one-hot encoding

In [ ]:
from tensorflow.keras.utils import to_categorical
class_n = len(set(train_y))
train_y = to_categorical(train_y, class_n)
test_y = to_categorical(test_y, class_n)

In [ ]:
train_x.shape, train_y.shape, test_x.shape

((14979, 28, 28, 1), (14979, 10), (3745, 28, 28, 1))

#### Sequential

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.backend import clear_session

In [ ]:
# 세션 클리어
clear_session()

# 모델 선언
model = Sequential()

# 모델 블록 조립
model.add(Input(shape=(28, 28, 1)))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(class_n, activation = 'softmax'))

# 컴파일
model.compile(loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'],
              optimizer = 'adam')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   verbose = 1)

In [ ]:
model.fit(train_x, train_y, epochs=10000, verbose=1,
          callbacks = [es], validation_split = 0.2)

Epoch 1/10000
375/375 [==============================] - 4s 6ms/step - loss: 0.6556 - accuracy: 0.8139 - val_loss: 0.4033 - val_accuracy: 0.8822
Epoch 2/10000
375/375 [==============================] - 2s 6ms/step - loss: 0.4113 - accuracy: 0.8774 - val_loss: 0.3981 - val_accuracy: 0.8842
Epoch 3/10000
375/375 [==============================] - 2s 5ms/step - loss: 0.3663 - accuracy: 0.8935 - val_loss: 0.3482 - val_accuracy: 0.8972
Epoch 4/10000
375/375 [==============================] - 2s 6ms/step - loss: 0.3240 - accuracy: 0.9059 - val_loss: 0.3590 - val_accuracy: 0.8992
Epoch 5/10000
375/375 [==============================] - 2s 6ms/step - loss: 0.2927 - accuracy: 0.9150 - val_loss: 0.3372 - val_accuracy: 0.9052
Epoch 6/10000
375/375 [==============================] - 2s 5ms/step - loss: 0.2672 - accuracy: 0.9175 - val_loss: 0.3505 - val_accuracy: 0.8962
Epoch 7/10000
375/375 [==============================] - 2s 5ms/step - loss: 0.2511 - accuracy: 0.9244 - val_loss: 0.3738 - val_ac

In [ ]:
model.evaluate(train_x, train_y)
model.evaluate(test_x, test_y)

118/118 [==============================] - 0s 2ms/step - loss: 0.3804 - accuracy: 0.9055


[0.380408376455307, 0.9054739475250244]

#### Functional

In [ ]:
# 클리어 세션
clear_session()

# 모델 엮기
il = Input(shape=(28, 28, 1))
fl = Flatten()(il)

hl = Dense(128, activation = 'relu')(fl)
hl = Dense(128, activation = 'relu')(hl)
bl = BatchNormalization()(hl)
dl = Dropout(0.2)(bl)

hl = Dense(64, activation = 'relu')(dl)
hl = Dense(64, activation = 'relu')(hl)
bl = BatchNormalization()(hl)
dl = Dropout(0.2)(bl)

ol = Dense(class_n, activation = 'softmax')(dl)

# 모델 input, output 정의하기
model = Model(il, ol)

# 모델 컴파일
model.compile(loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'],
              optimizer = 'adam')

In [ ]:
model.fit(train_x, train_y, epochs=10000, verbose=1,
          callbacks = [es], validation_split = 0.2)

Epoch 1/10000
375/375 [==============================] - 3s 4ms/step - loss: 0.7243 - accuracy: 0.7824 - val_loss: 0.4493 - val_accuracy: 0.8675
Epoch 2/10000
375/375 [==============================] - 1s 4ms/step - loss: 0.4437 - accuracy: 0.8706 - val_loss: 0.3972 - val_accuracy: 0.8815
Epoch 3/10000
375/375 [==============================] - 1s 3ms/step - loss: 0.3722 - accuracy: 0.8911 - val_loss: 0.3579 - val_accuracy: 0.8995
Epoch 4/10000
375/375 [==============================] - 1s 4ms/step - loss: 0.3372 - accuracy: 0.9015 - val_loss: 0.3499 - val_accuracy: 0.8982
Epoch 5/10000
375/375 [==============================] - 2s 5ms/step - loss: 0.3114 - accuracy: 0.9075 - val_loss: 0.3683 - val_accuracy: 0.8945
Epoch 6/10000
375/375 [==============================] - 1s 4ms/step - loss: 0.2857 - accuracy: 0.9150 - val_loss: 0.3449 - val_accuracy: 0.8985
Epoch 7/10000
375/375 [==============================] - 1s 4ms/step - loss: 0.2676 - accuracy: 0.9216 - val_loss: 0.3058 - val_ac

In [ ]:
model.evaluate(train_x, train_y)
model.evaluate(test_x, test_y)

118/118 [==============================] - 0s 2ms/step - loss: 0.3281 - accuracy: 0.9087


[0.32807907462120056, 0.9086782336235046]